<a href="https://colab.research.google.com/github/ccarpenterg/LearningMXNet/blob/master/03_introduction_to_convnets_with_mxnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction to Convolutional Neural Network with MXNet

In [0]:
!nvcc --version

In [0]:
!pip install mxnet-cu100

In [0]:
from __future__ import print_function

import mxnet as mx
from mxnet import nd, gluon, autograd
from mxnet.gluon import nn


print(mx.__version__)

1.5.0


### MNIST Dataset

In [0]:
def data_convention_normalization(data):
    return nd.moveaxis(data, 2, 0).astype('float32') / 255

train_data = gluon.data.vision.MNIST(train=True).transform_first(data_convention_normalization)
valid_data = gluon.data.vision.MNIST(train=False).transform_first(data_convention_normalization)

In [0]:
train_loader = gluon.data.DataLoader(train_data, shuffle=True, batch_size=64)
valid_loader = gluon.data.DataLoader(valid_data, shuffle=False, batch_size=64)

In [0]:
convnet = nn.Sequential()

convnet.add(nn.Conv2D(channels=32, kernel_size=3, activation='relu'),
        nn.MaxPool2D(pool_size=2),
        nn.Conv2D(channels=64, kernel_size=3, activation='relu'),
        nn.MaxPool2D(pool_size=2),
        nn.Conv2D(channels=64, kernel_size=3, activation='relu'),
        nn.Dense(64, activation='relu'),
        nn.Dense(10))

convnet

Sequential(
  (0): Conv2D(None -> 32, kernel_size=(3, 3), stride=(1, 1), Activation(relu))
  (1): MaxPool2D(size=(2, 2), stride=(2, 2), padding=(0, 0), ceil_mode=False, global_pool=False, pool_type=max, layout=NCHW)
  (2): Conv2D(None -> 64, kernel_size=(3, 3), stride=(1, 1), Activation(relu))
  (3): MaxPool2D(size=(2, 2), stride=(2, 2), padding=(0, 0), ceil_mode=False, global_pool=False, pool_type=max, layout=NCHW)
  (4): Conv2D(None -> 64, kernel_size=(3, 3), stride=(1, 1), Activation(relu))
  (5): Dense(None -> 64, Activation(relu))
  (6): Dense(None -> 10, linear)
)

In [0]:
ctx = mx.gpu(0) if mx.context.num_gpus() > 0 else mx.cpu(0)
convnet.initialize(mx.init.Xavier(), ctx=ctx)
convnet.summary(nd.zeros((1, 1, 28, 28), ctx=ctx))

--------------------------------------------------------------------------------
        Layer (type)                                Output Shape         Param #
               Input                              (1, 1, 28, 28)               0
        Activation-1                     <Symbol conv0_relu_fwd>               0
        Activation-2                             (1, 32, 26, 26)               0
            Conv2D-3                             (1, 32, 26, 26)             320
         MaxPool2D-4                             (1, 32, 13, 13)               0
        Activation-5                     <Symbol conv1_relu_fwd>               0
        Activation-6                             (1, 64, 11, 11)               0
            Conv2D-7                             (1, 64, 11, 11)           18496
         MaxPool2D-8                               (1, 64, 5, 5)               0
        Activation-9                     <Symbol conv2_relu_fwd>               0
       Activation-10        

In [0]:
trainer = gluon.Trainer(
    params=convnet.collect_params(),
    optimizer='sgd',
    optimizer_params={'learning_rate': 0.04},
)

In [0]:
metric = mx.metric.Accuracy()
loss_function = gluon.loss.SoftmaxCrossEntropyLoss()

In [0]:
num_epochs = 10

for epoch in range(num_epochs):
    for batch, labels in train_loader:
        batch = batch.as_in_context(ctx)
        labels = labels.as_in_context(ctx)
        
        with autograd.record():
            predictions = convnet(batch)
            loss = loss_function(predictions, labels)
            
        loss.backward()
        metric.update(labels, predictions)
        
        trainer.step(batch_size=batch.shape[0])
        
        name, acc = metric.get()
        
    print('Accuracy on epoch {}: {}'.format(epoch + 1, acc))
    metric.reset()
        

Accuracy on epoch 1: 0.8704166666666666
Accuracy on epoch 2: 0.9693
Accuracy on epoch 3: 0.97805
Accuracy on epoch 4: 0.9833833333333334
Accuracy on epoch 5: 0.9860833333333333
Accuracy on epoch 6: 0.9880833333333333
Accuracy on epoch 7: 0.98945
Accuracy on epoch 8: 0.9905
Accuracy on epoch 9: 0.9919166666666667
Accuracy on epoch 10: 0.9921666666666666


In [0]:
metric = mx.metric.Accuracy()

for batch, labels in valid_loader:
    batch = batch.as_in_context(ctx)
    labels = labels.as_in_context(ctx)
    metric.update(labels, convnet(batch))
    
print('Validation: {} = {}'.format(*metric.get()))